In [1]:
import numpy as np
import pandas as pd
# !pip install pgeocode
import warnings
warnings.simplefilter('ignore')
import pgeocode
nomi = pgeocode.Nominatim('IN')

In [2]:
invoice = pd.read_csv('E:\CodeChef\Capstone Project\Input_DataSet\Final_invoice.csv')
customer = pd.read_excel('E:\CodeChef\Capstone Project\Input_DataSet\Customer_Data.xlsx')
jtd = pd.read_csv('E:\CodeChef\Capstone Project\Input_DataSet\JTD.csv')

In [3]:
print("invoice data shape" , invoice.shape)
print("customer data shape" ,customer.shape)

invoice data shape (492314, 59)
customer data shape (555338, 9)


In [4]:
#droppinng records which have more than50% missing values
invoice = invoice.drop(['Amt Rcvd From Custom','Amt Rcvd From Ins Co',
                                          'CGST(14%)','CGST(2.5%)', 'CGST(6%)','IGST(12%)','CGST(9%)',
                                          'IGST(18%)','IGST(28%)','IGST(5%)','SGST/UGST(14%)',
                                          'SGST/UGST(2.5%)','SGST/UGST(6%)','SGST/UGST(9%)',
                                          'Total CGST','Total GST','Total IGST','Total SGST/UGST',
                                          'TDS amount','Service Advisor Name','Outstanding Amt',
                                          'Policy no.','Cash /Cashless Type','Expiry Date','Gate Pass Date',
                                          'Insurance Company','Claim No.'
                                         ],axis=1)

In [5]:
customer = customer.drop(['Date of Birth','Death date','Title','Marital Status','Occupation'],axis=1)

customer['Data Origin'] = customer['Data Origin'].fillna(customer['Data Origin'].mode()[0])
customer['Partner Type'] = customer['Partner Type'].fillna(customer['Partner Type'].mode()[0])

# Cleaning Geographic locations using Pgeocode Library

In [6]:
invoice["Clean_Location"]=nomi.query_postal_code(np.array(invoice["Pin code"].astype(str))).place_name
invoice["Clean_City"]=nomi.query_postal_code(np.array(invoice["Pin code"].astype(str))).county_name
invoice["Clean_State"]=nomi.query_postal_code(np.array(invoice["Pin code"].astype(str))).state_name

In [7]:
# dropping old Area,City,District after cleaning from the dataset
invoice.drop(["Area / Locality","CITY","District"],1,inplace=True)

# Deriving New Feature Service Time

In [8]:
invoice["JobCard_DateTime"]=invoice["JobCard Date"]+' '+invoice["JobCard Time"]
invoice["Invoice_DateTime"]=invoice["Invoice Date"]+' '+invoice["Invoice Time"]
invoice['JobCard_DateTime'] = pd.to_datetime(invoice['JobCard_DateTime'],dayfirst=True)
invoice['Invoice_DateTime'] = pd.to_datetime(invoice['Invoice_DateTime'],dayfirst=True)
invoice["Service_Time"]=invoice["Invoice_DateTime"]-invoice["JobCard_DateTime"]

In [9]:
#dropping old features of Invoice and Job Card Date and Time 
invoice.drop(['Invoice Date','Invoice Time','JobCard Date','JobCard Time'],axis = 1,inplace=True)

In [10]:
#Cleaned Customer and Invoice data
customer.to_csv('E:\CodeChef\Capstone Project\Cleaned DataSet\CleanedCustomer.csv')
invoice.to_csv('E:\CodeChef\Capstone Project\Cleaned DataSet\CleanedInvoice.csv')

In [12]:
#Reading Cleaned Customer and Invoice data for merging
customer_new = pd.read_csv('E:\CodeChef\Capstone Project\Cleaned DataSet\CleanedCustomer.csv',index_col = False)
invoice_new = pd.read_csv('E:\CodeChef\Capstone Project\Cleaned DataSet\CleanedInvoice.csv',index_col= False)

In [13]:
print(" jtd shape", jtd.shape)
print("invoice new shape" , invoice_new.shape)
print(" customer new shape" ,customer_new.shape)

 jtd shape (5619484, 10)
invoice new shape (492314, 32)
 customer new shape (555338, 5)


In [14]:
jtd.isnull().sum()

Unnamed: 0                   0
DBM Order                    0
Order Item                   0
Material                 51629
Labor Value Number     3774033
Description              51629
Item Category                0
Order Quantity               0
Target quantity UoM      51629
Net value                    0
dtype: int64

In [15]:
invoice_new.isnull().sum()

Unnamed: 0                 0
Unnamed: 0.1               0
Cust Type                  0
Customer No.               0
Gate Pass Time             0
Invoice No                 0
Job Card No                0
KMs Reading                0
Labour Total               0
Make                       0
Misc Total                 0
Model                   1525
ODN No.               197286
OSL Total                  0
Order Type                 0
Parts Total                0
Pin code                   0
Plant                      0
Plant Name1                0
Print Status               0
Recovrbl Exp               0
Regn No                   16
Technician Name       236834
Total Amt Wtd Tax.         0
Total Value           197286
User ID                    0
Clean_Location         47246
Clean_City             47246
Clean_State            47246
JobCard_DateTime           0
Invoice_DateTime           0
Service_Time               0
dtype: int64

In [16]:
#dropping  columns which have more missing values 
jtd.drop(['Labor Value Number','Unnamed: 0'],axis=1,inplace=True)
customer_new.drop(['Unnamed: 0'],axis=1,inplace=True)
invoice_new.drop(['ODN No.','Technician Name','Regn No','Unnamed: 0','Total Value'],axis=1,inplace=True)

In [17]:
jtd.head()

,DBM Order,Order Item,Material,Description,Item Category,Order Quantity,Target quantity UoM,Net value
0,22385,180,OSLCC_SRV31,MACHINING RR.SUSPENSION CHARGE (MAJOR),P010,1.2,MIN,1869.87
1,23910,10,1DM279018170117,ASSY.OIL COOLER (6-PLATE) (VOR,P002,1.0,EA,2151.13
2,23910,40,1IS150203BE01,FRONT WHEEL BEARING/ LUBRICATE,P002,1.0,EA,2552.92
3,23910,60,1DM279001990173,ASSY.CYL.HEAD W /O INJ.(NA W /O,P002,1.0,EA,20360.14
4,23910,70,1DM88552790S001SA,Engine Overhaul Kit -Indica Di,P002,1.0,EA,10691.63


In [18]:
#grouping the data in order to reduce duplicate entries
jtd_grouping=jtd.groupby(["DBM Order","Item Category"],as_index=False).agg({"Net value":'sum',"Order Quantity":"sum"})

In [19]:
jtd_grouping.shape

(1735310, 4)

In [20]:
jtd_grouping.head()

,DBM Order,Item Category,Net value,Order Quantity
0,4,P001,3455.92,714.0
1,4,P002,11042.62,15.6
2,4,P010,2399.95,1.0
3,4,P011,100.00,1.0
4,5,P001,2310.47,283.0


In [21]:
invoice_new.head()

,Unnamed: 0.1,Cust Type,Customer No.,Gate Pass Time,Invoice No,Job Card No,KMs Reading,Labour Total,Make,Misc Total,...,Print Status,Recovrbl Exp,Total Amt Wtd Tax.,User ID,Clean_Location,Clean_City,Clean_State,JobCard_DateTime,Invoice_DateTime,Service_Time
0,0,Retail,67849,00:00:00,7005200002,168303,49317,1203.14,GENERAL MOTORS,0.00,...,NO,0.0,4051.95,BC01FS1,"Thane Bazar, Thane H.O, Thane R.S.",Thane,Maharashtra,2014-12-13 14:29:43,2015-01-02 11:30:36,19 days 21:00:53.000000000
1,1,Retail,84419,00:00:00,7005200003,173997,78584,804.26,TATA MOTORS,197.03,...,NO,0.0,1001.29,BC01SA2,"Thane East, Kopri Colony",Thane,Maharashtra,2015-01-02 14:12:18,2015-01-03 10:07:32,0 days 19:55:14.000000000
2,2,Retail,81055,00:00:00,7005200004,173889,33985,180.19,MARUTI SUZUKI,0.00,...,NO,0.0,233.14,BC01SA2,"Chitalsar Manpada, Sandozbaugh",Thane,Maharashtra,2015-01-02 11:40:44,2015-01-03 11:12:57,0 days 23:32:13.000000000
3,3,Retail,84419,00:00:00,7005200005,174158,78584,0.00,TATA MOTORS,0.00,...,NO,0.0,0.00,BC01SA2,"Thane East, Kopri Colony",Thane,Maharashtra,2015-01-03 10:12:32,2015-01-03 11:40:44,0 days 01:28:12.000000000
4,4,Retail,18980,00:00:00,7005200006,173860,50057,1202.97,HYUNDAI,100.00,...,NO,0.0,2747.12,BC01SA2,"Thane East, Kopri Colony",Thane,Maharashtra,2015-01-02 10:45:42,2015-01-03 12:07:17,1 days 01:21:35.000000000


In [22]:
customer_new.head()

,Business Partner,Customer No.,Partner Type,Data Origin
0,31,1,1.0,Z001
1,32,2,1.0,Z005
2,34,4,1.0,Z005
3,35,5,1.0,Z005
4,42,7,1.0,Z008


In [23]:
#merging invoice and customer based on Customer No
inv_cust_merged = pd.merge(invoice_new,customer_new,how='left',on='Customer No.')

In [24]:
inv_cust_merged.head()

,Unnamed: 0.1,Cust Type,Customer No.,Gate Pass Time,Invoice No,Job Card No,KMs Reading,Labour Total,Make,Misc Total,...,User ID,Clean_Location,Clean_City,Clean_State,JobCard_DateTime,Invoice_DateTime,Service_Time,Business Partner,Partner Type,Data Origin
0,0,Retail,67849,00:00:00,7005200002,168303,49317,1203.14,GENERAL MOTORS,0.00,...,BC01FS1,"Thane Bazar, Thane H.O, Thane R.S.",Thane,Maharashtra,2014-12-13 14:29:43,2015-01-02 11:30:36,19 days 21:00:53.000000000,NaN,NaN,NaN
1,1,Retail,84419,00:00:00,7005200003,173997,78584,804.26,TATA MOTORS,197.03,...,BC01SA2,"Thane East, Kopri Colony",Thane,Maharashtra,2015-01-02 14:12:18,2015-01-03 10:07:32,0 days 19:55:14.000000000,NaN,NaN,NaN
2,2,Retail,81055,00:00:00,7005200004,173889,33985,180.19,MARUTI SUZUKI,0.00,...,BC01SA2,"Chitalsar Manpada, Sandozbaugh",Thane,Maharashtra,2015-01-02 11:40:44,2015-01-03 11:12:57,0 days 23:32:13.000000000,NaN,NaN,NaN
3,3,Retail,84419,00:00:00,7005200005,174158,78584,0.00,TATA MOTORS,0.00,...,BC01SA2,"Thane East, Kopri Colony",Thane,Maharashtra,2015-01-03 10:12:32,2015-01-03 11:40:44,0 days 01:28:12.000000000,NaN,NaN,NaN
4,4,Retail,18980,00:00:00,7005200006,173860,50057,1202.97,HYUNDAI,100.00,...,BC01SA2,"Thane East, Kopri Colony",Thane,Maharashtra,2015-01-02 10:45:42,2015-01-03 12:07:17,1 days 01:21:35.000000000,NaN,NaN,NaN


In [25]:
#Merging Invoice&Customer with JTD 
inv_cust_jtd_final=pd.merge(inv_cust_merged,jtd_grouping,how="left",left_on="Job Card No",right_on="DBM Order")

In [26]:
#Final Merged dataframe of Invoice,Customer and JTD 
inv_cust_jtd_final.shape

(936275, 34)

In [27]:
inv_cust_jtd_final.to_csv('E:\CodeChef\Capstone Project\Cleaned DataSet\CleanMergedData.csv')